In [37]:
import os
import pandas as pd
from tqdm import tqdm


import os
import re
import numpy as np
import pandas as pd
# import Shodan as sh
import torch
import torch.nn as nn
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import joblib
from IPython.display import display, HTML
import warnings
from sklearn.exceptions import InconsistentVersionWarning
import random
import time
import spacy

from utils import clean_add_info, remove_seniority_from_string, sanitize_text


warnings.filterwarnings("ignore", category=InconsistentVersionWarning)
warnings.filterwarnings("ignore", message=".*Torch was not compiled with flash attention.*", category=UserWarning)


# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
pd.set_option('display.max_columns', None)


# Define directories
results_dir = 'results_test'
os.makedirs(results_dir, exist_ok=True)
files_dir = 'data/dozagruzka/'
MODEL_DIR = "final_models"  # Directory where model files are stored


Using device: cuda


In [40]:

# -------------------- Load Models ----------------------
# Configuration



# Load tokenizer
print("Loading tokenizer...")
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
print("Tokenizer loaded successfully.")



# Define model classes
class ClassificationModel(nn.Module):
    """Base classification model for job titles using XLM-RoBERTa and categorical features"""
    def __init__(self, num_cat_features, num_classes, categorical_features, roberta_model_name='xlm-roberta-base'):
        super(ClassificationModel, self).__init__()
        self.roberta = XLMRobertaModel.from_pretrained(roberta_model_name)
        self.cat_embedding = nn.Embedding(num_cat_features, 50)
        self.fc1 = nn.Linear(self.roberta.config.hidden_size + 50 * len(categorical_features), 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU()
        self.categorical_features = categorical_features

    def forward(self, input_ids, attention_mask, cat_features):
        roberta_output = self.roberta(input_ids, attention_mask=attention_mask)
        roberta_output = roberta_output.pooler_output
        cat_features = self.cat_embedding(cat_features).view(cat_features.size(0), -1)
        x = torch.cat((roberta_output, cat_features), dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

class SpecializationModel(ClassificationModel):
    """Enhanced classification model for the specialization level with larger embeddings and dropout"""
    def __init__(self, num_cat_features, num_classes, categorical_features, roberta_model_name='xlm-roberta-base'):
        super(SpecializationModel, self).__init__(num_cat_features, num_classes, categorical_features, roberta_model_name)
        
        # Increased embedding dimension for categorical features
        embedding_dim = 64
        self.cat_embedding = nn.Embedding(num_cat_features, embedding_dim)
        
        # Increased hidden layer size
        hidden_size = 384
        self.fc1 = nn.Linear(self.roberta.config.hidden_size + embedding_dim * len(categorical_features), hidden_size)
        self.dropout = nn.Dropout(0.2)  # Added dropout for regularization
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask, cat_features):
        roberta_output = self.roberta(input_ids, attention_mask=attention_mask)
        roberta_output = roberta_output.pooler_output
        
        cat_features = self.cat_embedding(cat_features).view(cat_features.size(0), -1)
        
        x = torch.cat((roberta_output, cat_features), dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x



# Load tokenizer
print("Loading tokenizer...")
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
print("Tokenizer loaded successfully.")

# Define categorical features for each model level
categorical_features_1 = ['industry']
categorical_features_2 = ['industry', 'function']
categorical_features_3 = ['industry', 'function', 'subfunction']

# Enhanced function to load model with its specific encoders
def load_model_with_encoders(model_name, categorical_features, model_class=ClassificationModel):
    """
    Load a model and its associated encoders dynamically
    
    Parameters:
    model_name (str): Base name of the model (e.g., 'Function', 'Subfunction', 'Spec')
    categorical_features (list): List of categorical feature names
    model_class (nn.Module): The model class to instantiate
    
    Returns:
    tuple: (model, encoders_dict) - The loaded model and a dictionary of its encoders
    """
    print(f"\nLoading {model_name} model and its encoders...")
    model_path = os.path.join(MODEL_DIR, f"{model_name}_final.pth")
    encoders_dict = {}
    
    try:
        # First load state dict to get dimensions
        state_dict = torch.load(model_path, map_location=device)
        num_cat_features = state_dict['cat_embedding.weight'].shape[0]
        embedding_dim = state_dict['cat_embedding.weight'].shape[1]
        
        # Load all associated encoders for this model
        for feature in categorical_features:
            # Special case for target (output) encoder
            if feature == 'target':
                encoder_path = os.path.join(MODEL_DIR, f"{model_name}_le_target.pkl")
            else:
                encoder_path = os.path.join(MODEL_DIR, f"{model_name}_le_{feature}.pkl")
            
            # Try to load model-specific encoder, fall back to generic if needed
            if os.path.exists(encoder_path):
                encoders_dict[feature] = joblib.load(encoder_path)
                print(f"  - Loaded {feature} encoder for {model_name} model")
            else:
                # Look for generic encoder
                generic_path = os.path.join(MODEL_DIR, f"Function_le_{feature}.pkl")
                if os.path.exists(generic_path):
                    encoders_dict[feature] = joblib.load(generic_path)
                    print(f"  - Used generic {feature} encoder for {model_name} model")
                else:
                    raise FileNotFoundError(f"Could not find encoder for {feature}")
        
        # Load output encoder (always model-specific)
        target_path = os.path.join(MODEL_DIR, f"{model_name}_le_target.pkl")
        encoders_dict['target'] = joblib.load(target_path)
        print(f"  - Loaded target encoder for {model_name} model")
        
        # Get number of output classes
        num_classes = len(encoders_dict['target'].classes_)
        
        # Create and load model with correct dimensions
        print(f"  - Creating {model_name} model with {num_cat_features} categorical features and {num_classes} output classes")
        model = model_class(num_cat_features, num_classes, categorical_features).to(device)
        model.load_state_dict(state_dict)
        model.eval()
        
        return model, encoders_dict
        
    except Exception as e:
        print(f"Error loading {model_name} model: {e}")
        raise

# Load all models and their encoders
try:
    # Function model
    model_function, function_encoders = load_model_with_encoders(
        'Function', 
        categorical_features_1
    )
    
    # Subfunction model
    model_subfunction, subfunction_encoders = load_model_with_encoders(
        'Subfunction', 
        categorical_features_2
    )
    
    # Specialization model
    model_specialization, specialization_encoders = load_model_with_encoders(
        'Spec', 
        categorical_features_3,
        model_class=SpecializationModel
    )
    
    print("\nAll models and encoders loaded successfully!")
    
    # Create combined industry mapping
    industry_mapping = {}
    # Use industry encoder from Function model as the base
    for industry in function_encoders['industry'].classes_:
        # Create multiple entries with different formats for the same industry
        cleaned = sanitize_text(industry)
        industry_mapping[cleaned] = industry
        # Add the original version too
        industry_mapping[industry] = industry
        # Add a version with capitalization but no special chars
        simplified = re.sub(r'[^a-zA-Zа-яА-ЯёЁ0-9\s]', '', industry)
        industry_mapping[simplified] = industry
        industry_mapping[simplified.lower()] = industry
        # Add version with special chars converted to spaces
        spacified = re.sub(r'[^a-zA-Zа-яА-ЯёЁ0-9]', ' ', industry).strip()
        industry_mapping[spacified] = industry
        industry_mapping[spacified.lower()] = industry
    
except Exception as e:
    print(f"Error in model loading process: {e}")
    raise


# Enhanced prediction functions for each level with their specific encoders and confidence scores
def enhanced_predict_function(model, tokenizer, text, industry, function_encoders):
    """Predict function category for a job title with confidence score"""
    inputs = tokenizer.encode_plus(text, add_special_tokens=True, max_length=128, padding='max_length', 
                                  return_attention_mask=True, return_tensors='pt', truncation=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    
    # Encode industry using function model's specific encoder
    encoded_industry = function_encoders['industry'].transform([industry])[0]
    industry_tensor = torch.tensor([encoded_industry], dtype=torch.long).to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, cat_features=industry_tensor)
    
    # Apply softmax to get probabilities
    probabilities = torch.nn.functional.softmax(outputs, dim=1)
    
    # Get predicted class and its probability
    pred_idx = outputs.argmax(dim=1).item()
    confidence = probabilities[0, pred_idx].item()
    
    return pred_idx, function_encoders['target'].inverse_transform([pred_idx])[0], confidence

def enhanced_predict_subfunction(model, tokenizer, text, industry, function, subfunction_encoders):
    """Predict subfunction category for a job title with confidence score"""
    inputs = tokenizer.encode_plus(text, add_special_tokens=True, max_length=128, padding='max_length', 
                                  return_attention_mask=True, return_tensors='pt', truncation=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    
    # Encode industry and function using subfunction model's specific encoders
    encoded_industry = subfunction_encoders['industry'].transform([industry])[0]
    encoded_function = subfunction_encoders['function'].transform([function])[0]
    
    features_tensor = torch.tensor([[encoded_industry, encoded_function]], dtype=torch.long).to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, cat_features=features_tensor)
    
    # Apply softmax to get probabilities
    probabilities = torch.nn.functional.softmax(outputs, dim=1)
    
    # Get predicted class and its probability
    pred_idx = outputs.argmax(dim=1).item()
    confidence = probabilities[0, pred_idx].item()
    
    return pred_idx, subfunction_encoders['target'].inverse_transform([pred_idx])[0], confidence

def enhanced_predict_specialization(model, tokenizer, text, industry, function, subfunction, specialization_encoders):
    """Predict specialization category for a job title with confidence score"""
    inputs = tokenizer.encode_plus(text, add_special_tokens=True, max_length=128, padding='max_length', 
                                  return_attention_mask=True, return_tensors='pt', truncation=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    
    # Encode industry, function and subfunction using specialization model's specific encoders
    encoded_industry = specialization_encoders['industry'].transform([industry])[0]
    encoded_function = specialization_encoders['function'].transform([function])[0]
    encoded_subfunction = specialization_encoders['subfunction'].transform([subfunction])[0]
    
    features_tensor = torch.tensor([[encoded_industry, encoded_function, encoded_subfunction]], dtype=torch.long).to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, cat_features=features_tensor)
    
    # Apply softmax to get probabilities
    probabilities = torch.nn.functional.softmax(outputs, dim=1)
    
    # Get predicted class and its probability
    pred_idx = outputs.argmax(dim=1).item()
    confidence = probabilities[0, pred_idx].item()
    
    return pred_idx, specialization_encoders['target'].inverse_transform([pred_idx])[0], confidence

def select_p_features(df):
    df_divs = df.iloc[:, 2:8]
    last_two_divs = []
    # print("ok")
    for row, _ in tqdm(df_divs.iterrows()):
        # print("row ", df.iloc[row, :], "col", cols)
        # break
        only_divisions = df_divs.iloc[row, :]
        only_divisions.dropna(inplace=True)
        result = []
        # print("ok")
        for p in only_divisions:
            # print(p)
            p_cleaned = clean_add_info(p)
            
            # print("cleaned: ", p_cleaned)
            # если все слова в p_cleaned содержатся где-то в result, то мы не должны добавлять этот p_cleaned к result
            if (p_cleaned!=''):
                result+=[p_cleaned]
        if len(result)>1:
            last_two_divs += [result[-2] + ' ' + result[-1]]
        elif len(result)==1:
            last_two_divs += [result[-1]]
        else:
            last_two_divs += ['']
    return last_two_divs
        # print(only_divisions)



Loading tokenizer...
Tokenizer loaded successfully.
Loading tokenizer...
Tokenizer loaded successfully.

Loading Function model and its encoders...
  - Loaded industry encoder for Function model
  - Loaded target encoder for Function model
  - Creating Function model with 22 categorical features and 100 output classes

Loading Subfunction model and its encoders...
  - Loaded industry encoder for Subfunction model
  - Loaded function encoder for Subfunction model
  - Loaded target encoder for Subfunction model
  - Creating Subfunction model with 123 categorical features and 565 output classes

Loading Spec model and its encoders...
  - Loaded industry encoder for Spec model
  - Loaded function encoder for Spec model
  - Loaded subfunction encoder for Spec model
  - Loaded target encoder for Spec model
  - Creating Spec model with 688 categorical features and 231 output classes

All models and encoders loaded successfully!


In [41]:
def predict(df, matched_industry):
    unique_job_titles = df['text_input'].drop_duplicates().tolist()
    print(f"Found {len(unique_job_titles)} unique job titles to process")

    # Dictionary to store predictions for unique job titles
    job_predictions = {}

    # Process each unique job title
    for job_title in tqdm(unique_job_titles, desc="Predicting unique job titles"):
        try:
            # print("ok")
            # Clean job title
            # job_title_clean = sanitize_text(job_title)
            job_title_clean = remove_seniority_from_string(job_title)
            # print(job_title_clean)
            
            # Predict function with confidence
            _, function_name, function_confidence = enhanced_predict_function(
                model_function, tokenizer, job_title_clean, matched_industry, function_encoders
            )
            
            # Predict subfunction with confidence
            _, subfunction_name, subfunction_confidence = enhanced_predict_subfunction(
                model_subfunction, tokenizer, job_title_clean, matched_industry, 
                function_name, subfunction_encoders
            )
            
            # Predict specialization with confidence
            _, specialization_name, specialization_confidence = enhanced_predict_specialization(
                model_specialization, tokenizer, job_title_clean, matched_industry, 
                function_name, subfunction_name, specialization_encoders
            )
            
            # Store predictions and confidence scores for this job title
            job_predictions[job_title] = {
                'function_pred': function_name,
                'function_confidence': function_confidence,
                'subfunction_pred': subfunction_name,
                'subfunction_confidence': subfunction_confidence,
                'specialization_pred': specialization_name if specialization_name != '-' else None,
                'specialization_confidence': specialization_confidence if specialization_name != '-' else None
            }
            
        except Exception as e:
            print(f"Error processing '{job_title}': {str(e)}")
            job_predictions[job_title] = {
                'function_pred': f"ERROR: {str(e)}",
                'function_confidence': None
            }
    return job_predictions

In [42]:


#--------------------- Predict -------------------------
# Single function to process all files
def process_files_and_predict():
    """
    Function that loops through files in files_dir, 
    makes predictions, and saves results to results_dir.
    
    - Each file has one industry
    - We process only unique job titles to avoid redundant processing
    - Results are mapped back to all occurrences
    - Now includes confidence scores for each prediction
    """
    print(f"Looking for files in: {files_dir}")
    print(f"Results will be saved to: {results_dir}")
    company_naming = 0
    # Loop through all Excel files in the directory
    for filename in os.listdir(files_dir):
        if not filename.endswith(('.xlsx', '.xls', '.xlsm')):
            file_path = os.path.join(files_dir, filename)
            print(f"\nProcessing {filename}...")

            # sh.shodan_speak()
            
            # Load file
            try:
                # df = pd.read_excel(file_path, sheet_name='Данные', header=6)
                df = pd.read_parquet(file_path)
                # df = df.iloc[:, :-35]
                df= df.iloc[:500, :]
                df.rename(columns={'Подразделение 1 уровня': 'p1', 'Подразделение 2 уровня': 'p2','Подразделение 3 уровня': 'p3','Подразделение 4 уровня': 'p4','Подразделение 5 уровня': 'p5','Подразделение 6 уровня': 'p6', 'Код функции': 'function', 'Сектор': 'industry', 'Название должности':'job_title', 'Название функции (заполняется автоматически)': 'function_name'}, inplace=True)
                df.drop(columns=['function_name'], inplace=True)
                # df = df.iloc[:500,:]
                # df_info = pd.read_excel(file_path, sheet_name='Общая информация', header=1)
                print(f"Loaded file with shape: {df.shape}")
            except Exception as e:
                print(f"Error loading {filename}: {e}")
                continue
            
            # Check required columns
            if 'job_title' not in df.columns:
                print(f"Required column 'job_title' not found in {filename}. Skipping file.")
                continue
                
            # Find matching industry from encoders
            matched_industry = None
            
            # Try exact match first
            if industry in function_encoders['industry'].classes_:
                matched_industry = industry
            else:
                # Try case-insensitive match
                industry_lower = industry.lower()
                for ind in function_encoders['industry'].classes_:
                    if ind.lower() == industry_lower:
                        matched_industry = ind
                        break
                        
                # If still not found, try partial match
                if not matched_industry:
                    for ind in function_encoders['industry'].classes_:
                        if industry_lower in ind.lower() or ind.lower() in industry_lower:
                            matched_industry = ind
                            break
            
            if not matched_industry:
                print(f"Warning: Industry '{industry}' not found in encoders")
                print(f"Available industries: {function_encoders['industry'].classes_}")
                print(f"Skipping file {filename} as industry cannot be matched")
                continue
                
            print(f"Matched industry: '{industry}' → '{matched_industry}'")

            
            
            # df = df.reset_index(drop=True) 
            if 'add_info' in df.columns:
                df.drop(columns='add_info', inplace=True)
            if 'text_input' in df.columns:
                df.drop(columns='text_input',inplace=True)
            
            lst = select_p_features(df)
            res_col = pd.Series(lst, name='add_info')
            df = pd.concat([df, res_col], axis=1)
            print(df.columns)
            # print(df.index.is_unique)

            # # print(df)

            df = df.reset_index(drop=True) 

            df["text_input"] = (
                df["job_title"].astype(str) +
                " [SEP] " + df["add_info"].astype(str)
            )

            # Get unique job titles
            job_predictions = predict(df, matched_industry)
            
            # Apply predictions to the DataFrame
            print("Mapping predictions back to all occurrences...")
            
            # Initialize new columns
            df['function_pred'] = None
            df['function_confidence'] = None
            # df['subfunction'] = None
            # df['subfunction_confidence'] = None
            # df['specialization'] = None
            # df['specialization_confidence'] = None
            
            # Map the predictions back to all rows
            for job_title, predictions in tqdm(job_predictions.items()):
                # Find all rows with this job title
                mask = df['text_input'] == job_title
                # Apply predictions and confidence scores
                for col, value in predictions.items():
                    df.loc[mask, col] = value
            
            # # Format confidence scores as percentages (optional)
            for col in ['function_confidence', 'subfunction_confidence', 'specialization_confidence']:
                mask = df[col].notna()
                if mask.any():
                    df.loc[mask, col] = df.loc[mask, col].apply(lambda x: f"{x:.2%}")
            
            # Save results
            output_path = os.path.join(results_dir, f"{company_naming}_FunPreds_NoDeps_mymodel.xlsx")
            company_naming +=1
            df.to_excel(output_path, index=False)
            print(f"Saved results to {output_path}")
    
    print(f"\nAll files processed!")

In [43]:
# 2. Process all files in the directory
process_files_and_predict()


Looking for files in: data/dozagruzka/
Results will be saved to: results_test

Processing Rawdata_2025_before_reload.parquet.gzip...
Loaded file with shape: (500, 65)
Matched industry: 'Энергетика' → 'Энергетика'


500it [00:03, 128.76it/s]


Index(['Название компании (заполняется автоматически)', 'p1', 'p2', 'p3', 'p4',
       'p5', 'p6', 'job_title', 'Код сотрудника',
       'Код руководителя сотрудника', 'Руководитель / специалист',
       'Оценка эффективности работы сотрудника',
       'Уровень подчинения по отношению к Первому лицу компании', 'Экспат',
       'Пол', 'Год рождения', 'Дата приема на работу',
       'Сотрудники, проработавшие в компании меньше 1 года', 'Название города',
       'Регион/область (заполняется автоматически)',
       'Внутренний грейд компании', 'Грейд / Уровень обзора', 'function',
       'Код подфункции', 'Код специализации',
       'Название подфункции (заполняется автоматически)',
       'Название специализации (заполняется автоматически)', 'Размер ставки',
       'Ежемесячный оклад', 'Число окладов в году',
       'Постоянные надбавки и доплаты (общая сумма за год)',
       'Право на получение переменного вознаграждения', 'Фактическая премия',
       'Целевая премия (%)',
       'Право 

Predicting unique job titles: 100%|██████████| 233/233 [00:03<00:00, 67.67it/s]


Mapping predictions back to all occurrences...


100%|██████████| 233/233 [00:00<00:00, 2138.40it/s]
C:\Users\sivch\AppData\Local\Temp\ipykernel_24196\3371223070.py:122: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['90.68%' '99.94%' '99.33%' '78.43%' '98.94%' '59.51%' '97.74%' '97.74%'
 '38.44%' '96.36%' '100.00%' '46.91%' '91.17%' '91.17%' '91.17%' '99.94%'
 '38.78%' '88.47%' '84.08%' '43.40%' '43.40%' '98.94%' '98.94%' '79.55%'
 '99.35%' '99.35%' '99.35%' '99.12%' '99.12%' '99.99%' '98.94%' '98.94%'
 '98.94%' '98.94%' '98.94%' '98.94%' '99.82%' '99.98%' '99.97%' '100.00%'
 '100.00%' '40.53%' '99.15%' '99.98%' '54.49%' '98.36%' '98.36%' '93.65%'
 '40.71%' '99.94%' '99.94%' '99.94%' '99.94%' '99.85%' '96.02%' '96.02%'
 '40.29%' '40.29%' '40.29%' '100.00%' '96.84%' '99.96%' '100.00%'
 '100.00%' '100.00%' '100.00%' '100.00%' '94.46%' '40.53%' '99.65%'
 '99.65%' '50.42%' '50.42%' '50.42%' '99.90%' '64.16%' '94.13%' '94.13%'
 '94.13%' '98.57%' '98.57%' '

Saved results to results_test\0_FunPreds_NoDeps_mymodel.xlsx

All files processed!


In [44]:
result2 = pd.read_excel("results_test/0_FunPreds_NoDeps_mymodel.xlsx")
result2

,Название компании (заполняется автоматически),p1,p2,p3,p4,p5,p6,job_title,Код сотрудника,Код руководителя сотрудника,Руководитель / специалист,Оценка эффективности работы сотрудника,Уровень подчинения по отношению к Первому лицу компании,Экспат,Пол,Год рождения,Дата приема на работу,"Сотрудники, проработавшие в компании меньше 1 года",Название города,Регион/область (заполняется автоматически),Внутренний грейд компании,Грейд / Уровень обзора,function,Код подфункции,Код специализации,Название подфункции (заполняется автоматически),Название специализации (заполняется автоматически),Размер ставки,Ежемесячный оклад,Число окладов в году,Постоянные надбавки и доплаты (общая сумма за год),Право на получение переменного вознаграждения,Фактическая премия,Целевая премия (%),Право на участие в Программе долгосрочного вознаграждения (LTIP),Фактическая стоимость всех предоставленных типов LTI за 1 год,Целевая стоимость всех предоставленных типов LTI в % от базового оклада за 1 год,Тип программы 1,Фактическая стоимость вознаграждения 1 за 1 год,Целевая стоимость вознаграждения 1 как % от базового оклада за 1 год,Частота выплат 1,Тип программы 2,Фактическая стоимость вознаграждения 2 за 1 год,Целевая стоимость вознаграждения 2 как % от базового оклада за 1 год,Частота выплат 2,Тип программы 3,Фактическая стоимость вознаграждения 3 за 1 год,Целевая стоимость вознаграждения 3 как % от базового оклада за 1 год,Частота выплат 3,Комментарии,Макрорегион,industry,Тип компании,Общее количество сотрудников по состоянию на 1 мая 2025 года,"Выручка за 2024 год, руб.",Годовой оклад (AP),Базовый оклад (BP),Краткосрочное фактическое переменное вознаграждение (VP),Целевая Премия (TI),Фактическое совокупное вознаграждение (TC),Целевое совокупное вознаграждение (TTC),Фактическое долгосрочное вознаграждение (LTIP),Целевое долгосрочное вознаграждение (TLTIP),Прямое совокупное вознаграждение (TDC),Целевое прямое совокупное вознаграждение (TTDC),add_info,text_input,function_pred,function_confidence,subfunction_pred,subfunction_confidence,specialization_pred,specialization_confidence
0,AliasGroup,Основное подразделение,Блок проектирования (в строительстве),Старшее звено управления,NaN,NaN,NaN,Директор департамента проектирования / Генерал...,41,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,Москва,Москва,NaN,20,EG,EGO,EGO-A,Инженерное проектирование,Инженерное проектирование,1,NaN,12,NaN,Нет,NaN,NaN,Нет,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Центральный регион,Строительство и недвижимость,Локальная компания,1. ≤ 500,"2. 3,3 - 33 млрд руб.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,старшее звено управления директор проектирован...,Директор департамента проектирования / Генерал...,EG,99.81%,EGO,90.68%,EGO-B,86.28%
1,AliasGroup,Коммерческий департамент,Блок продаж,Старшее звено управления,NaN,NaN,NaN,Коммерческий Директор / Директор по продажам,2790,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,Москва,Москва,NaN,20,SL,SLZ,NaN,Директор по продажам,-,1,NaN,12,NaN,Да,2.160000e+07,2.57143,Нет,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Центральный регион,Строительство и недвижимость,Локальная компания,1. ≤ 500,"2. 3,3 - 33 млрд руб.",NaN,NaN,NaN,2.57143,NaN,NaN,NaN,NaN,NaN,NaN,старшее звено управления коммерческий директор...,Коммерческий Директор / Директор по продажам [...,SL,99.94%,SLZ,99.94%,NaN,NaN
2,AliasGroup,Управление по информационным технологиям,Блок информационных технологий (разработка и в...,Старшее звено управления,NaN,NaN,NaN,Директор Службы Информационных Технологий (СIO...,2,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,Москва,Москва,NaN,20,IT,ITZ,NaN,ИТ Директор,-,1,NaN,12,NaN,Да,1.311840e+07,0.89000,Нет,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Центральный регион,Строительство и недвижимость,Локальная компания,1. ≤ 500,"2. 3,3 - 33 млрд руб.",NaN,NaN,NaN,0.89000,NaN,NaN,NaN,NaN,NaN,NaN,старшее звено управления директор информационн...,Директор Службы Информационных Технологий (СIO...,IT,99.97%,ITZ,99.33%,NaN,NaN
3,AliasGroup,Департаме

In [31]:
df = pd.read_parquet('data/dozagruzka/Rawdata_2025_before_reload.parquet.gzip')

df = df.iloc[:, :-35]
df.rename(columns={'Подразделение 1 уровня': 'p1', 'Подразделение 2 уровня': 'p2','Подразделение 3 уровня': 'p3','Подразделение 4 уровня': 'p4','Подразделение 5 уровня': 'p5','Подразделение 6 уровня': 'p6', 'Код функции': 'function', 'Сектор': 'industry', 'Название должности':'job_title', 'Название функции (заполняется автоматически)': 'function_name'}, inplace=True)
df.drop(columns=['function_name'], inplace=True)
df

,Название компании (заполняется автоматически),p1,p2,p3,p4,p5,p6,job_title,Код сотрудника,Код руководителя сотрудника,Руководитель / специалист,Оценка эффективности работы сотрудника,Уровень подчинения по отношению к Первому лицу компании,Экспат,Пол,Год рождения,Дата приема на работу,"Сотрудники, проработавшие в компании меньше 1 года",Название города,Регион/область (заполняется автоматически),Внутренний грейд компании,Грейд / Уровень обзора,function,Код подфункции,Код специализации,Название подфункции (заполняется автоматически),Название специализации (заполняется автоматически),Размер ставки,Ежемесячный оклад,Число окладов в году
0,AliasGroup,Основное подразделение,Блок проектирования (в строительстве),Старшее звено управления,None,None,None,Директор департамента проектирования / Генерал...,41,None,None,-,None,None,None,None,None,None,Москва,Москва,None,20,EG,EGO,EGO-A,Инженерное проектирование,Инженерное проектирование,1.0,NaN,12.0
1,AliasGroup,Коммерческий департамент,Блок продаж,Старшее звено управления,None,None,None,Коммерческий Директор / Директор по продажам,2790,None,None,-,None,None,None,None,None,None,Москва,Москва,None,20,SL,SLZ,None,Директор по продажам,-,1.0,NaN,12.0
2,AliasGroup,Управление по информационным технологиям,Блок информационных технологий (разработка и в...,Старшее звено управления,None,None,None,Директор Службы Информационных Технологий (СIO...,2,None,None,-,None,None,None,None,None,None,Москва,Москва,None,20,IT,ITZ,None,ИТ Директор,-,1.0,NaN,12.0
3,AliasGroup,Департамент по внешним коммуникациям,Блок по связям с общественностью,Старшее звено управления,None,None,None,Директор по связям с общественностью,16108,None,None,-,None,None,None,None,None,None,Москва,Москва,None,18,CA,CAC,CAC-A,Связи с общественностью (PR),Пресс-служба и связи с общественностью,1.0,NaN,12.0
4,AliasGroup,Дирекция по персоналу,Блок по управлению персоналом,Старшее звено управления,None,None,None,Директор по персоналу (численность < 1500 сотр...,16,None,None,-,None,None,None,None,None,None,Москва,Москва,None,19,HR,HRZ,None,Директор по управлению персоналом,-,1.0,NaN,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37686,TPS_Nedvizhimost,Филиал «Хорошо!» АО «ТПС Недвижимость»/ Дирекц...,Технический / производственный блок,Среднее звено управления,PRD_0400,Начальник отдела по эксплуатации и техобслужив...,None,Заместитель директора филиала по эксплуатации ...,474,None,None,-,None,None,None,None,None,None,Москва,Москва,None,16,EG,EGA,None,Инженер общего профиля,-,1.0,NaN,12.0
37687,TPS_Nedvizhimost,Филиал «Хорошо!» АО «ТПС Недвижимость»/ Дирекц...,Блок проектирования (в строительстве),Специалисты,DEV_0320,Ведущий инженер по слаботочным / пожарным сист...,None,Инженер по слаботочным системам,475,None,None,-,None,None,None,None,None,None,Москва,Москва,None,14,EG,EGA,None,Инженер общего профиля,-,1.0,NaN,12.0
37688,TPS_Nedvizhimost,Филиал «Хорошо!» АО «ТПС Недвижимость»/ Управл...,Блок по безопасности,Среднее звено управления,SED_0050,Начальник службы охраны,None,Руководитель управления,476,None,None,-,None,None,None,None,None,None,Москва,Москва,None,15,SE,SEB,None,Охрана объектов,-,1.0,NaN,12.0
37689,TPS_Nedvizhimost,Филиал «Хорошо!» АО «ТПС Недвижимость»/ Управл...,Блок по безопасности,Специалисты,SED_0120,Ведущий специалист по региональной безопасности,None,Ведущий специалист,477,None,None,-,None,None,None,None,None,None,Москва,Москва,None,13,SE,SEB,None,Охрана объектов,-,1.0,NaN,12.0


In [46]:
errors = result2.loc[result2['function']!=result2['function_pred']]
errors

,Название компании (заполняется автоматически),p1,p2,p3,p4,p5,p6,job_title,Код сотрудника,Код руководителя сотрудника,Руководитель / специалист,Оценка эффективности работы сотрудника,Уровень подчинения по отношению к Первому лицу компании,Экспат,Пол,Год рождения,Дата приема на работу,"Сотрудники, проработавшие в компании меньше 1 года",Название города,Регион/область (заполняется автоматически),Внутренний грейд компании,Грейд / Уровень обзора,function,Код подфункции,Код специализации,Название подфункции (заполняется автоматически),Название специализации (заполняется автоматически),Размер ставки,Ежемесячный оклад,Число окладов в году,Постоянные надбавки и доплаты (общая сумма за год),Право на получение переменного вознаграждения,Фактическая премия,Целевая премия (%),Право на участие в Программе долгосрочного вознаграждения (LTIP),Фактическая стоимость всех предоставленных типов LTI за 1 год,Целевая стоимость всех предоставленных типов LTI в % от базового оклада за 1 год,Тип программы 1,Фактическая стоимость вознаграждения 1 за 1 год,Целевая стоимость вознаграждения 1 как % от базового оклада за 1 год,Частота выплат 1,Тип программы 2,Фактическая стоимость вознаграждения 2 за 1 год,Целевая стоимость вознаграждения 2 как % от базового оклада за 1 год,Частота выплат 2,Тип программы 3,Фактическая стоимость вознаграждения 3 за 1 год,Целевая стоимость вознаграждения 3 как % от базового оклада за 1 год,Частота выплат 3,Комментарии,Макрорегион,industry,Тип компании,Общее количество сотрудников по состоянию на 1 мая 2025 года,"Выручка за 2024 год, руб.",Годовой оклад (AP),Базовый оклад (BP),Краткосрочное фактическое переменное вознаграждение (VP),Целевая Премия (TI),Фактическое совокупное вознаграждение (TC),Целевое совокупное вознаграждение (TTC),Фактическое долгосрочное вознаграждение (LTIP),Целевое долгосрочное вознаграждение (TLTIP),Прямое совокупное вознаграждение (TDC),Целевое прямое совокупное вознаграждение (TTDC),add_info,text_input,function_pred,function_confidence,subfunction_pred,subfunction_confidence,specialization_pred,specialization_confidence
27,AliasGroup,Архитектурно-конструкторская группа,Блок проектирования (в строительстве),Среднее звено управления,NaN,NaN,NaN,Главный инженер-конструктор,153,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,Москва,Москва,NaN,16,EG,EGO,EGO-A,Инженерное проектирование,Инженерное проектирование,1,NaN,12,NaN,Да,1032000.000,0.30000,Нет,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Центральный регион,Строительство и недвижимость,Локальная компания,1. ≤ 500,"2. 3,3 - 33 млрд руб.",NaN,NaN,NaN,0.30000,NaN,NaN,NaN,NaN,NaN,NaN,среднее звено управления главный инженер - кон...,Главный инженер-конструктор [SEP] среднее звен...,RD,99.69%,RDE,99.12%,NaN,NaN
28,AliasGroup,Архитектурно-конструкторская группа,Блок проектирования (в строительстве),Среднее звено управления,NaN,NaN,NaN,Главный инженер-конструктор,74,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,Москва,Москва,NaN,16,EG,EGO,EGO-A,Инженерное проектирование,Инженерное проектирование,1,NaN,12,NaN,Да,1446000.000,0.43000,Нет,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Центральный регион,Строительство и недвижимость,Локальная компания,1. ≤ 500,"2. 3,3 - 33 млрд руб.",NaN,NaN,NaN,0.43000,NaN,NaN,NaN,NaN,NaN,NaN,среднее звено управления главный инженер - кон...,Главный инженер-конструктор [SEP] среднее звен...,RD,99.69%,RDE,99.12%,NaN,NaN
30,AliasGroup,Архитектурно-конструкторская группа,Блок проектирования (в строительстве),Среднее звено управления,NaN,NaN,NaN,Начальник отдела по предпроектной подготовке,17,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,Москва,Москва,NaN,16,EG,EGO,EGO-A,Инженерное проектирование,Инженерное проектирование,1,NaN,12,NaN,Да,1860000.000,0.43000,Нет,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Центральный регион,Строительство и недвижимость,Локальная компания,1. ≤ 500,"2. 3,3 - 33 млрд руб.",NaN,NaN,NaN,0.43000,NaN,NaN,NaN,NaN,NaN,NaN,среднее звено управления начальник предпроектн...,Начальник отдела по предпроект

In [35]:

errors = result2.loc[result2['function']!=result2['function_pred']]
errors

,Название компании (заполняется автоматически),p1,p2,p3,p4,p5,p6,job_title,Код сотрудника,Код руководителя сотрудника,Руководитель / специалист,Оценка эффективности работы сотрудника,Уровень подчинения по отношению к Первому лицу компании,Экспат,Пол,Год рождения,Дата приема на работу,"Сотрудники, проработавшие в компании меньше 1 года",Название города,Регион/область (заполняется автоматически),Внутренний грейд компании,Грейд / Уровень обзора,function,Код подфункции,Код специализации,Название подфункции (заполняется автоматически),Название специализации (заполняется автоматически),Размер ставки,Ежемесячный оклад,Число окладов в году,add_info,text_input,function_pred,function_confidence
27,AliasGroup,Архитектурно-конструкторская группа,Блок проектирования (в строительстве),Среднее звено управления,NaN,NaN,NaN,Главный инженер-конструктор,153,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,Москва,Москва,NaN,16,EG,EGO,EGO-A,Инженерное проектирование,Инженерное проектирование,1.0,NaN,12,среднее звено управления главный инженер - кон...,Главный инженер-конструктор [SEP] среднее звен...,RD,0.996853
28,AliasGroup,Архитектурно-конструкторская группа,Блок проектирования (в строительстве),Среднее звено управления,NaN,NaN,NaN,Главный инженер-конструктор,74,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,Москва,Москва,NaN,16,EG,EGO,EGO-A,Инженерное проектирование,Инженерное проектирование,1.0,NaN,12,среднее звено управления главный инженер - кон...,Главный инженер-конструктор [SEP] среднее звен...,RD,0.996853
30,AliasGroup,Архитектурно-конструкторская группа,Блок проектирования (в строительстве),Среднее звено управления,NaN,NaN,NaN,Начальник отдела по предпроектной подготовке,17,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,Москва,Москва,NaN,16,EG,EGO,EGO-A,Инженерное проектирование,Инженерное проектирование,1.0,NaN,12,среднее звено управления начальник предпроектн...,Начальник отдела по предпроектной подготовке [...,CU,0.971189
31,AliasGroup,Инженерная группа внутренних систем,Блок проектирования (в строительстве),Среднее звено управления,NaN,NaN,NaN,Начальник отдела по предпроектной подготовке,21,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,Москва,Москва,NaN,16,EG,EGO,EGO-A,Инженерное проектирование,Инженерное проектирование,1.0,NaN,12,среднее звено управления начальник предпроектн...,Начальник отдела по предпроектной подготовке [...,CU,0.971189
32,AliasGroup,Служба технического заказчика,Блок проектирования (в строительстве),Среднее звено управления,NaN,NaN,NaN,Начальник отдела по предпроектной подготовке,137,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,Москва,Москва,NaN,16,EG,EGS,NaN,Технический надзор,-,1.0,NaN,12,среднее звено управления начальник предпроектн...,Начальник отдела по предпроектной подготовке [...,CU,0.971189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37669,TPS_Nedvizhimost,Филиал «Хорошо!» АО «ТПС Недвижимость»/ Дирекц...,Технический / производственный блок,Рабочие / Вспомогательный персонал,PRD_0160,Механик,NaN,Дежурный инженер,457,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,Москва,Москва,NaN,12,PR,PRB,PRB-E,Рабочие должности,Механосборочные работы,1.0,NaN,12,механик дежурный инженер,Дежурный инженер [SEP] механик дежурный инженер,EG,0.969319
37670,TPS_Nedvizhimost,Филиал «Хорошо!» АО «ТПС Недвижимость»/ Дирекц...,Технический / производственный блок,Рабочие / Вспомогательный персонал,PRD_0160,Механик,NaN,Дежурный инженер,458,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,Москва,Москва,NaN,12,PR,PRB,PRB-E,Рабочие должности,Механосборочные работы,1.0,NaN,12,механик дежурный инженер,Дежурный инженер [SEP] механик дежурный инженер,EG,0.969319
37677,TPS_Nedvizhimost,Филиал «Хорошо!» АО «ТПС Недвижимость»/ Дирекц...,Технический / производственный блок,Рабочие / Вспомогательный персонал,PRD_0482,Электромеханик по сетям,NaN,Специалист по слаботочным системам,465,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,Москва,Москва,NaN,12,EG,EGE,NaN,Инженер-электрик,-,1.0,NaN,12,электромеханик сетям специалист слаботочным си...,Специалис